In [ ]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = {}
markers = ['RASIS', 'LASIS', 'RPSIS', 'LPSIS', 'RTH1', 'RTH2', 'RTH3', 'RTH4', 'RFLE', 'RFAL', 'RSK1', 'RSK2', 'RSK4', 'RSK3', 'RFCC', 'RFMT1', 'RFMT2', 'RFMT5', 'LTH1', 'LTH2', 'LTH3', 'LTH4', 'LFLE', 'LSK1', 'LSK2', 'LSK4', 'LSK3', 'LFAL', 'LFCC', 'LFMT1', 'LFMT2', 'LFMT5']

In [ ]:

def extract_ik_weights(ik, markers = markers):

    info = {}
    # Print the full structure
    for file in ik:
        a=0
        b=0
        weight = {}
        tree = ET.parse(file)
        root = tree.getroot()
        while root[0][b].tag != 'IKTaskSet':
            b+=1
        while True:                
            try:
                if (root[0][b][0][a].attrib)["name"] in markers:
                    weight[(root[0][b][0][a].attrib)["name"]] = {root[0][b][0][a][1].tag:root[0][b][0][a][1].text,
                                                                        root[0][b][0][a][0].tag:root[0][b][0][a][0].text}
                a+=1
            except IndexError:
                break
            
        info[Path(file).name] = weight
    return info

In [ ]:

def plot_ik_weights_heatmap(info,subject):
    # 1. Transform your dictionary into a long-form DataFrame
    rows = []
    for trial, markers in info.items():
        trial_id = trial.replace('.xml', '').replace('ik_setup_', '')
        for marker, details in markers.items():
            rows.append({
                'Trial': trial_id,
                'Marker': marker,
                'Weight': float(details['weight'])
            })

    df = pd.DataFrame(rows)

    # 2. Pivot for the Heatmap
    pivot_df = df.pivot(index='Marker', columns='Trial', values='Weight')
    # pivot_df = pivot_df[['STW1', 'STW2', 'STW3', 'STW4', 'STW5']] # Ensure order

    # 3. Plotting the Heatmap
    plt.figure(figsize=(10, 12))
    sns.heatmap(pivot_df, annot=True, fmt='.0f', cmap='YlGnBu', cbar_kws={'label': 'Weight'})
    plt.title('Distribution of Marker Weights across IK Trials')
    plt.savefig(f'marker_weights_heatmaps{subject}.png')

    # 4. Plotting a Comparison for Key Markers
    # key_markers = ['RASIS', 'RPSIS', 'RTH1', 'RFLE', 'RFCC']
    # subset_df = df[df['Marker'].isin(key_markers)]

    # plt.figure(figsize=(10, 6))
    # sns.barplot(data=subset_df, x='Marker', y='Weight', hue='Trial')
    # plt.title('Comparison of Specific Marker Weights')
    # plt.ylabel('Weight Value')
    # plt.savefig('key_markers_weights_bar_s02.png')

In [ ]:

def plot_ik_weights_comparison(data):
    # Assuming 'data' contains subjects as top-level keys
    # Example format: {'Subject_01': {...}, 'Subject_02': {...}}

    rows = []
    for subject, setups in data.items():
        for setup_name, markers in setups.items():
            for marker, details in markers.items():
                rows.append({
                    'Subject': subject,
                    'Marker': marker,
                    'Weight': float(details['weight'])
                })

    df_comp = pd.DataFrame(rows)

    # Create a Box Plot to see the variance per marker across all subjects
    plt.figure(figsize=(14, 8))
    sns.boxplot(data=df_comp, x='Marker', y='Weight', palette='Set3')
    sns.stripplot(data=df_comp, x='Marker', y='Weight', color='black', alpha=0.3) # Show individual points
    plt.xticks(rotation=45)
    plt.title('Comparison of Marker Weights Across All Subjects')
    plt.ylabel('Weight Value')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('marker_weights_comparison_all_subjects.png')
    plt.show()

In [ ]:

for subject in range(2,11):
    ik = [rf"d:\UG_Proj\Human Sitting to Walking Transitions\S{subject:02d}\IK\ik_setup_stw{trial}.xml" for trial in range(1,6)]
    info = extract_ik_weights(ik)
    # plot_ik_weights_heatmap(info,subject)
    data[f'Subject_{subject:02d}'] = info
    
    

In [ ]:
data.keys()

In [ ]:
plot_ik_weights_comparison(data)

In [ ]:
data["Subject_07"]["ik_setup_stw1.xml"]

In [ ]:
import opensim as osim
import os

In [ ]:
os.chdir(r"d:\UG_Proj\Human Sitting to Walking Transitions\S02")

In [ ]:
iktool = osim.InverseKinematicsTool(r"d:\UG_Proj\Human Sitting to Walking Transitions\S02\IK\ik_setup_STW1.xml")

In [ ]:
iktool.get_marker_file()

In [ ]:
iktool.run()

In [ ]:
from opensim import InverseKinematicsTool, Model
import os


os.chdir(r"d:\UG_Proj\Human Sitting to Walking Transitions\S02\IK") #already present
# Check current working directory
print(f"Current working directory: {os.getcwd()}")

# Load IK setup
ik_setup = "d:\\UG_Proj\\Human Sitting to Walking Transitions\\S02\\IK\\ik_setup_STW1.xml"
ik_tool = InverseKinematicsTool(ik_setup)

# Get the model file path it's using
model_file = ik_tool.get_model_file()
print(f"Model file (relative): {model_file}")

# Resolve to absolute path
abs_model_path = os.path.abspath(model_file)
print(f"Model file (absolute): {abs_model_path}")

# Check if file exists
if os.path.exists(abs_model_path):
    print(f"✓ File found: {abs_model_path}")
else:
    print(f"✗ File NOT found: {abs_model_path}")

# Run IK
# ik_tool.run()

In [ ]:
print("IK process completed.")

In [ ]:
import opensim as osm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- 1. Setup Paths and Model ---
model_path = r'd:\UG_Proj\Human Sitting to Walking Transitions\S01\scale\subject01_simbody_scaled.osim'
ik_file = r'd:\UG_Proj\Human Sitting to Walking Transitions\S01\IK\results_stw\ik_output_s01_stw1.mot' # Replace with your IK file name
output_dir = './analysis_results'

model = osm.Model(model_path)
model.initSystem()

# --- 2. Define Point Kinematics Analysis ---
# We assume the RLCC and LLCC are the markers you want to track.
# If they are not in the model, we use the center of the calcaneus bodies.
bodies_to_analyze = ['calcn_r', 'calcn_l']
marker_names = ['RFCC', 'LFCC']

# Create the PointKinematics analysis
pk = osm.PointKinematics(model)
pk.setStartTime(0) # Adjust based on your IK data start time
pk.setEndTime(5)  # Adjust based on your IK data end time

# For each marker, we define its body and local position
# Here we use 0,0,0 as the local origin of the calcaneus bodies
for (body_name,marker_name) in (bodies_to_analyze,marker_names):
    pk.setBody(model.getBodySet().get(body_name))
    pk.setPoint(osm.Vec3(0, 0, 0)) # Point in local body frame
    pk.setPointName(marker_name)

model.addAnalysis(pk)

# --- 3. Run Analysis Tool ---
analyze = osm.AnalyzeTool()
analyze.setModel(model)
analyze.setStatesFileName(ik_file)
analyze.setResultsDir(output_dir)
analyze.run()

print("Point Acceleration analysis complete. Files saved in:", output_dir)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

def calculate_marker_acceleration(trc_path, output_csv='marker_accelerations.csv', cutoff_freq=6.0):
    # 1. Load TRC file
    # We read the headers carefully to find RFCC and LFCC
    with open(trc_path, 'r') as f:
        lines = f.readlines()
    
    # Marker names are usually on line 3, components (X, Y, Z) on line 4
    marker_names = lines[3].split('\t')
    print(marker_names)
    
    # Find the column index for RFCC and LFCC
    # TRC format: Frame (0), Time (1), M1_X (2), M1_Y (3), M1_Z (4)...
    rfcc_idx = -1
    lfcc_idx = -1
    
    current_col = 2
    for m in marker_names:
        m = m.strip()
        if m == 'RFCC': rfcc_idx = current_col
        if m == 'LFCC': lfcc_idx = current_col
        if m: current_col += 3 # Each marker has 3 columns (X, Y, Z)

    if rfcc_idx == -1 or lfcc_idx == -1:
        raise ValueError("RFCC or LFCC markers not found in the TRC file.")

    # 2. Load the numerical data
    data = pd.read_csv(trc_path, sep='\t', skiprows=5, header=None)
    time = data.iloc[:, 1].values
    fs = 1.0 / np.mean(np.diff(time)) # Sampling frequency
    
    # Extract X, Y, Z for both markers
    rfcc_pos = data.iloc[:, rfcc_idx:rfcc_idx+3].values
    lfcc_pos = data.iloc[:, lfcc_idx:lfcc_idx+3].values

    # 3. Filter the Position Data (Butterworth Low-pass)
    # This is mandatory; differentiating raw noise creates huge errors.
    b, a = butter(4, cutoff_freq / (fs / 2), btype='low')
    rfcc_filt = filtfilt(b, a, rfcc_pos, axis=0)
    lfcc_filt = filtfilt(b, a, lfcc_pos, axis=0)

    # 4. Differentiate twice to get Acceleration (m/s^2)
    # Velocity
    rfcc_vel = np.gradient(rfcc_filt, time, axis=0)
    lfcc_vel = np.gradient(lfcc_filt, time, axis=0)
    # Acceleration
    rfcc_acc = np.gradient(rfcc_vel, time, axis=0)
    lfcc_acc = np.gradient(lfcc_vel, time, axis=0)

    # 5. Determine which leg steps first
    # We look for the first peak in Vertical (Y) acceleration (index 1)
    # A threshold of 0.5 m/s^2 is usually enough to detect movement
    threshold = 2
    r_start_idx = np.where(np.abs(rfcc_acc[:, 1]) > threshold)[0][0]
    l_start_idx = np.where(np.abs(lfcc_acc[:, 1]) > threshold)[0][0]
    
    first_leg = "Right (RFCC)" if r_start_idx < l_start_idx else "Left (LFCC)"
    print(f"The {first_leg} leg starts moving first at {time[min(r_start_idx, l_start_idx)]:.3f}s")

    # 6. Save and Plot
    results = pd.DataFrame({
        'time': time,
        'RFCC_Acc_Y': rfcc_acc[:, 1],
        'LFCC_Acc_Y': lfcc_acc[:, 1]
    })
    results.to_csv(output_csv, index=False)
    
    plt.figure(figsize=(10, 5))
    plt.plot(time, rfcc_acc[:, 1], label='RFCC (Right) Vertical Accel')
    plt.plot(time, lfcc_acc[:, 1], label='LFCC (Left) Vertical Accel')
    plt.axvline(time[r_start_idx], color='blue', linestyle='--', alpha=0.5, label='R-Start')
    plt.axvline(time[l_start_idx], color='red', linestyle='--', alpha=0.5, label='L-Start')
    plt.title('Heel Marker Vertical Acceleration')
    plt.ylabel('Acceleration ($m/s^2$)')
    plt.xlabel('Time (s)')
    plt.legend()
    plt.savefig('heel_acceleration_plot.png')
    
    return results

# To run:
dfs = calculate_marker_acceleration(r'd:\UG_Proj\Human Sitting to Walking Transitions\S02\ExpData\Mocap\trcResults\stw3.trc')

In [ ]:
import xml.etree.ElementTree as ET

def extract_muscle_max_force(osim_xml_path):
    """
    Reads an OpenSim XML (.osim) file and returns a list of
    (muscle_name, max_isometric_force).
    """
    tree = ET.parse(osim_xml_path)
    root = tree.getroot()

    muscles = []

    # Iterate over all Thelen2003Muscle elements
    for muscle in root.iter('Thelen2003Muscle'):
        name = muscle.attrib.get('name', 'UNKNOWN')

        max_force_elem = muscle.find('max_isometric_force')
        if max_force_elem is not None:
            max_iso = float(max_force_elem.text)
            minimum_activation = muscle.findtext('minimum_activation')

        muscles.append({
            "name": name,
            "max_isometric_force": float(max_iso) if max_iso else None,
            "minimum_activation": float(minimum_activation) if minimum_activation else None
        })

    return muscles


# Example usage
osim_file = r"c:\Users\Ritesh Aditya\Documents\OpenSim\4.5\Models\Gait2392_Simbody\gait2392_simbody.osim"
muscle_list = extract_muscle_max_force(osim_file)

for muscle in muscle_list:
    print(f"{muscle['name']:20s}  {muscle['max_isometric_force']:8.1f} N {muscle['minimum_activation']} m")
import csv
with open("muscle_max_forces_gait2392.csv", "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["name", "max_isometric_force", "minimum_activation"]
    )
    writer.writeheader()
    writer.writerows(muscle_list)

In [ ]:
import xml.etree.ElementTree as ET

def extract_muscle_max_force(osim_xml_path):
    """
    Reads an OpenSim XML (.osim) file and returns a list of
    (muscle_name, max_isometric_force).
    """
    tree = ET.parse(osim_xml_path)
    root = tree.getroot()

    muscles = []

    # Iterate over all Thelen2003Muscle elements
    for muscle in root.iter('Millard2012EquilibriumMuscle'):
        name = muscle.attrib.get('name', 'UNKNOWN')

        max_force_elem = muscle.find('max_isometric_force')
        if max_force_elem is not None:
            max_iso = float(max_force_elem.text)
            minimum_activation = muscle.findtext('minimum_activation')

        muscles.append({
            "name": name,
            "max_isometric_force": float(max_iso) if max_iso else None,
            "minimum_activation": float(minimum_activation) if minimum_activation else None
        })

    return muscles


# Example usage
osim_file = r"d:\student\MTech\Sakshi\STW\model\RajagopalLa2023_LL-stw_adjustedWieghts.osim"
muscle_list = extract_muscle_max_force(osim_file)
print("Muscle Max Isometric Forces from RajagopalLa2023_LL-stw_adjustedWieghts.osim:")
for muscle in muscle_list:
    print(f"{muscle['name']:20s}  {muscle['max_isometric_force']:8.1f} N {muscle['minimum_activation']} m")


import csv
with open("muscle_max_forces_rajagopal.csv", "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["name", "max_isometric_force", "minimum_activation"]
    )
    writer.writeheader()
    writer.writerows(muscle_list)

In [ ]:
import xml.etree.ElementTree as ET

def extract_muscle_max_force(osim_xml_path):
    """
    Reads an OpenSim XML (.osim) file and returns a list of
    (muscle_name, max_isometric_force).
    """
    tree = ET.parse(osim_xml_path)
    root = tree.getroot()

    muscles = []

    # Iterate over all Thelen2003Muscle elements
    for muscle in root.iter('Millard2012EquilibriumMuscle'):
        name = muscle.attrib.get('name', 'UNKNOWN')

        max_force_elem = muscle.find('max_isometric_force')
        if max_force_elem is not None:
            max_iso = float(max_force_elem.text)
            minimum_activation = muscle.findtext('minimum_activation')

        muscles.append({
            "name": name,
            "max_isometric_force": float(max_iso) if max_iso else None,
            "minimum_activation": float(minimum_activation) if minimum_activation else None
        })

    return muscles


# Example usage
osim_file = r"d:\student\MTech\Sakshi\STW\S01\scale\subject01_simbody_scaled.osim"
muscle_list_1 = extract_muscle_max_force(osim_file)
print("Muscle Max Isometric Forces from subject01_simbody_scaled.osim:")
for muscle in muscle_list:
    print(f"{muscle['name']:20s}  {muscle['max_isometric_force']:8.1f} N {muscle['minimum_activation']} m")


import csv
with open("muscle_max_forces_rajagopal.csv", "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["name", "max_isometric_force", "minimum_activation"]
    )
    writer.writeheader()
    writer.writerows(muscle_list)

In [ ]:
muscle_list_1 == muscle_list

In [ ]:
import opensim as osm

# --- CONFIGURATION ---
input_model_file = r"d:\student\MTech\Sakshi\STW\S01\scale\subject01_simbody_scaled.osim"
output_model_file = r"d:\student\MTech\Sakshi\STW\S01\scale\subject01_simbody_scaled_stronger.osim"
strength_scale_factor = 1.5  # Increase strength by 50%

# Load the model
model = osm.Model(input_model_file)
muscle_set = model.getMuscles()

print(f"Scaling {muscle_set.getSize()} muscles by a factor of {strength_scale_factor}...")

# Iterate through muscles and multiply max_isometric_force
for i in range(muscle_set.getSize()):
    muscle = muscle_set.get(i)
    current_force = muscle.get_max_isometric_force()
    new_force = current_force * strength_scale_factor
    muscle.set_max_isometric_force(new_force)

# Save the modified model
model.printToXML(output_model_file)

print(f"Success! New model saved as {output_model_file}")

# smoothing grf

In [ ]:
import re
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

def read_mot_with_header(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    # find endheader index
    header_idx = next((i for i,l in enumerate(lines) if 'endheader' in l.lower()), None)
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")
    header_lines = lines[:header_idx+1]      # include the 'endheader' line
    colnames_line = lines[header_idx+1].rstrip("\n")
    # load data (pandas will use the next line as column names)
    df = pd.read_csv(path, sep=r'\s+', engine='python', skiprows=header_idx+1)
    return header_lines, colnames_line, df

def detect_y_columns(df):
    cols = []
    for c in df.columns:
        low = c.lower()
        if re.search(r'(_vy$|_fy$|vy$|fy$)', low) or ('ground_force' in low and 'vy' in low):
            cols.append(c)
    return cols

def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    wn = cutoff / nyq
    b, a = butter(order, wn, btype='low')
    return filtfilt(b, a, data, axis=0)

def filter_and_save_mot(in_path,
                        out_path=None,
                        cutoff=20.0,
                        order=4,
                        fs=None,
                        fs_fallback=1000.0,
                        columns=None,
                        plot=False):
    """
    Read `in_path` (.mot/.sto), low-pass filter Y-GRF columns, and save to `out_path`
    preserving the original header and column-label line.
    If out_path is None, a new file `<inname>_filtered.mot` is created next to input.
    """
    header_lines, colnames_line, df = read_mot_with_header(in_path)

    if columns is None:
        columns = detect_y_columns(df)
    if not columns:
        raise ValueError("No Y-direction GRF columns detected. Provide `columns=` explicitly.")

    # estimate fs if not provided
    time_col = next((c for c in df.columns if c.lower() == 'time'), df.columns[0])
    time = df[time_col].values
    if fs is None:
        if len(time) >= 2:
            dt = np.mean(np.diff(time))
            fs = 1.0 / dt
        else:
            fs = fs_fallback

    # prepare numeric matrix and filter
    data_mat = df[columns].astype(float).values
    filtered_mat = butter_lowpass_filter(data_mat, cutoff=cutoff, fs=fs, order=order)

    # place filtered values back into dataframe copy
    df_filt = df.copy()
    df_filt.loc[:, columns] = filtered_mat

    # choose output path
    if out_path is None:
        base, ext = os.path.splitext(in_path)
        out_path = base + '_filtered' + (ext if ext else '.mot')

    # write header + original column label line + filtered data (no pandas header)
    with open(out_path, 'w', newline='\n') as f:
        f.writelines(header_lines)
        f.write(colnames_line + '\n')
    # append data without header (tab-separated for readability)
    df_filt.to_csv(out_path, sep='\t', index=False, header=False, float_format='%.6f', mode='a')
    print(f"Saved filtered .mot to: {out_path}")

    if plot:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10,6))
        for c in columns:
            plt.plot(time, df_filt[c].values, label=c)
        plt.xlabel('Time (s)')
        plt.ylabel('Force (N)')
        plt.title('Filtered GRF Y-direction (combined)')
        plt.legend(fontsize='small')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    return df_filt, out_path

# Example usage:
in_mot = r'd:\student\MTech\test\Mocap\grfResults\stw4.mot'
df_filtered, out_file = filter_and_save_mot(in_mot, cutoff=20, order=4, plot=True)

In [ ]:
import opensim as osim
import os

os.chdir(r"D:\student\MTech\test") #already present
so_analysis = osim.AnalyzeTool("so_setup_s01_stw4.xml")
so_analysis.setResultsDir(r"D:\student\MTech\test\output")
so_analysis.run()

muscle force filtered

In [ ]:
import re
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

def read_mot_with_header(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    # find endheader index
    header_idx = next((i for i,l in enumerate(lines) if 'endheader' in l.lower()), None)
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")
    header_lines = lines[:header_idx+1]      # include the 'endheader' line
    colnames_line = lines[header_idx+1].rstrip("\n")
    # load data (pandas will use the next line as column names)
    df = pd.read_csv(path, sep=r'\s+', engine='python', skiprows=header_idx+1)
    return header_lines, colnames_line, df

def detect_y_columns(df):
    cols = []
    for c in df.columns:
        low = c.lower()
        if re.search(r'(_vy$|_fy$|vy$|fy$)', low) or ('ground_force' in low and 'vy' in low):
            cols.append(c)
    return cols

def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    wn = cutoff / nyq
    b, a = butter(order, wn, btype='low')
    return filtfilt(b, a, data, axis=0)

def filter_and_save_mot(in_path,
                        out_path=None,
                        cutoff=20.0,
                        order=4,
                        fs=None,
                        fs_fallback=250.0,
                        columns=None,
                        plot=False):
    """
    Read `in_path` (.mot/.sto), low-pass filter Y-GRF columns, and save to `out_path`
    preserving the original header and column-label line.
    If out_path is None, a new file `<inname>_filtered.mot` is created next to input.
    """
    header_lines, colnames_line, df = read_mot_with_header(in_path)
    print(f"columnnames: {df.columns.tolist()}")

    if columns is None:
        columns = detect_y_columns(df)
    if not columns:
        raise ValueError("No Y-direction GRF columns detected. Provide `columns=` explicitly.")

    # estimate fs if not provided
    time_col = next((c for c in df.columns if c.lower() == 'time'), df.columns[0])
    time = df[time_col].values
    if fs is None:
        if len(time) >= 2:
            dt = np.mean(np.diff(time))
            fs = 1.0 / dt
        else:
            fs = fs_fallback
    columne = df.columns.tolist()
    # prepare numeric matrix and filter
    data_mat = df[columne].astype(float).values
    filtered_mat = butter_lowpass_filter(data_mat, cutoff=cutoff, fs=fs, order=order)

    # place filtered values back into dataframe copy
    df_filt = df.copy()
    df_filt.loc[:, columne] = filtered_mat

    # choose output path
    if out_path is None:
        base, ext = os.path.splitext(in_path)
        out_path = base + '_filtered' + (ext if ext else '.mot')

    # write header + original column label line + filtered data (no pandas header)
    with open(out_path, 'w', newline='\n') as f:
        f.writelines(header_lines)
        f.write(colnames_line + '\n')
    # append data without header (tab-separated for readability)
    df_filt.to_csv(out_path, sep='\t', index=False, header=False, float_format='%.6f', mode='a')
    print(f"Saved filtered .mot to: {out_path}")

    if plot:
        import matplotlib.pyplot as plt
        import math
        # create grid of subplots
        n = len(columns)
        if n == 0:
            print('No columns to plot')
        else:
            ncols = min(3, n)
            nrows = math.ceil(n / ncols)
            fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 4, max(2.5, nrows * 2.5)), sharex=True)

            # robustly flatten axes into a list of Axes objects
            if isinstance(axes, np.ndarray):
                axes_flat = axes.ravel().tolist()
            elif isinstance(axes, (list, tuple)):
                axes_flat = []
                for row in axes:
                    if isinstance(row, (list, tuple, np.ndarray)):
                        axes_flat.extend(list(row))
                    else:
                        axes_flat.append(row)
            else:
                axes_flat = [axes]

            for ax, c in zip(axes_flat, columns):
                ax.plot(time, df_filt[c].values)
                ax.set_xlabel('Time (s)')
                ax.set_ylabel(f'{c} Force (N)')
                ax.set_title(f'Filtered muscle forces [{c}]')
                ax.grid(True)

            # hide any unused axes
            for ax in axes_flat[len(columns):]:
                try:
                    ax.set_visible(False)
                except Exception:
                    pass

            plt.tight_layout()
            plt.savefig(f'filtered_muscle_forces_{os.path.basename(out_path)}')
            plt.show()

    return df_filt, out_path


in_mot = r'D:\student\MTech\test\output\subject01_StaticOptimization_force.sto'
df_filtered, out_file = filter_and_save_mot(in_mot, cutoff=20, order=4, plot=True, columns=['bflh_r','bfsh_r','recfem_r','semimem_r','semiten_r','soleus_r','tibant_r','tibpost_r','vasint_r','vaslat_r','vasmed_r'])
# df_filtered, out_file = filter_and_save_mot(in_mot, cutoff=20, order=4, plot=True, columns= ['bflh_l','bfsh_l','gaslat_l','gasmed_l','glmax1_l','glmax2_l','glmax3_l','glmed1_l','glmed2_l','glmed3_l','recfem_l','semimem_l','semiten_l','soleus_l','tibant_l','tibpost_l','vasint_l','vaslat_l','vasmed_l'])

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


def read_mot_with_header(path):
    """
    Read an OpenSim .mot/.sto file while preserving header and column labels.
    """
    with open(path, 'r') as f:
        lines = f.readlines()

    header_idx = next(
        (i for i, l in enumerate(lines) if 'endheader' in l.lower()),
        None
    )
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")

    header_lines = lines[:header_idx + 1]
    colnames_line = lines[header_idx + 1].rstrip("\n")

    df = pd.read_csv(
        path,
        sep=r'\s+',
        engine='python',
        skiprows=header_idx + 1
    )

    return header_lines, colnames_line, df


def plot_mot_without_filtering(
    in_path,
    out_path=None,
    columns=None,
    plot=True
):
    """
    Read an OpenSim .mot/.sto file, DO NOT FILTER anything,
    optionally save a copy, and plot selected columns.
    """

    header_lines, colnames_line, df = read_mot_with_header(in_path)

    if columns is None or len(columns) == 0:
        raise ValueError("Please provide columns to plot")

    # Time column detection
    time_col = next(
        (c for c in df.columns if c.lower() == 'time'),
        df.columns[0]
    )
    time = df[time_col].values

    # Optional save (unchanged data)
    if out_path is not None:
        with open(out_path, 'w', newline='\n') as f:
            f.writelines(header_lines)
            f.write(colnames_line + '\n')

        df.to_csv(
            out_path,
            sep='\t',
            index=False,
            header=False,
            float_format='%.6f',
            mode='a'
        )

        print(f"Saved unfiltered file to: {out_path}")

    # -------- PLOTTING --------
    if plot:
        n = len(columns)
        ncols = min(3, n)
        nrows = math.ceil(n / ncols)

        fig, axes = plt.subplots(
            nrows=nrows,
            ncols=ncols,
            figsize=(ncols * 4, max(2.5, nrows * 2.5)),
            sharex=True
        )

        # Flatten axes safely
        if isinstance(axes, np.ndarray):
            axes_flat = axes.ravel().tolist()
        else:
            axes_flat = [axes]

        for ax, c in zip(axes_flat, columns):
            ax.plot(time, df[c].values)
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Force (N)')
            ax.set_title(f'Muscle force [{c}]')
            ax.grid(True)

        # Hide unused subplots
        for ax in axes_flat[len(columns):]:
            ax.set_visible(False)

        plt.tight_layout()
        plt.savefig(
            f'raw_muscle_forces_{os.path.basename(in_path)}.png',
            dpi=300
        )
        plt.show()

    return df


# ============================
# USAGE
# ============================

in_mot = r'd:\student\MTech\test\output\subject01_StaticOptimization_force.sto'

df_raw = plot_mot_without_filtering(
    in_mot,
    plot=True,
    columns=[
        'bflh_r',
        'bfsh_r',
        'recfem_r',
        'semimem_r',
        'semiten_r',
        'soleus_r',
        'tibant_r',
        'tibpost_r',
        'vasint_r',
        'vaslat_r',
        'vasmed_r'
    ]
)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


def read_mot_with_header(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    header_idx = next(
        (i for i, l in enumerate(lines) if 'endheader' in l.lower()),
        None
    )
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")

    header_lines = lines[:header_idx + 1]
    colnames_line = lines[header_idx + 1].rstrip("\n")

    df = pd.read_csv(
        path,
        sep=r'\s+',
        engine='python',
        skiprows=header_idx + 1
    )

    return header_lines, colnames_line, df


def get_all_muscle_columns(df):
    """
    In Static Optimization force .sto files,
    everything except 'time' is a muscle force.
    """
    return [c for c in df.columns if c.lower() != 'time']


def plot_all_muscles_percent(
    in_path,
    out_path=None,
    plot=True
):
    """
    Plot ALL muscle forces vs 0–100% movement cycle
    (no filtering).
    """

    header_lines, colnames_line, df = read_mot_with_header(in_path)

    # ---- Time normalization to 0–100% ----
    time_col = next(
        (c for c in df.columns if c.lower() == 'time'),
        df.columns[0]
    )
    time = df[time_col].values
    percent = 100 * (time - time[0]) / (time[-1] - time[0])

    # ---- Muscle detection ----
    muscle_columns = get_all_muscle_columns(df)
    print(f"Found {len(muscle_columns)} muscles")

    # Optional save (unchanged data)
    if out_path is not None:
        with open(out_path, 'w', newline='\n') as f:
            f.writelines(header_lines)
            f.write(colnames_line + '\n')

        df.to_csv(
            out_path,
            sep='\t',
            index=False,
            header=False,
            float_format='%.6f',
            mode='a'
        )
        print(f"Saved unfiltered file to: {out_path}")

    # ---------------- PLOTTING ----------------
    if plot:
        n = len(muscle_columns)
        ncols = 4
        nrows = math.ceil(n / ncols)

        fig, axes = plt.subplots(
            nrows=nrows,
            ncols=ncols,
            figsize=(ncols * 4, nrows * 2.2),
            sharex=True
        )

        axes = np.atleast_1d(axes).ravel()

        for ax, muscle in zip(axes, muscle_columns):
            ax.plot(percent, df[muscle].values, linewidth=1)
            ax.set_title(muscle, fontsize=9)
            ax.set_ylabel('Force (N)')
            ax.grid(True)

        for ax in axes[len(muscle_columns):]:
            ax.set_visible(False)

        for ax in axes[:ncols]:
            ax.set_xlabel('Movement cycle (%)')

        plt.xlim(0, 100)
        plt.tight_layout()
        plt.savefig(
            f'all_muscle_forces_0_100pct_{os.path.basename(in_path)}.png',
            dpi=300
        )
        plt.show()

    return df


# ============================
# USAGE
# ============================

in_mot = r'D:\student\MTech\test\output\subject01_StaticOptimization_force.sto'

df_raw = plot_all_muscles_percent(
    in_mot,
    plot=True
)


muscle filtered plot for all muscles



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.signal import butter, filtfilt


# -------------------------------------------------
# Read OpenSim .sto/.mot with header preserved
# -------------------------------------------------
def read_mot_with_header(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    header_idx = next(
        (i for i, l in enumerate(lines) if 'endheader' in l.lower()),
        None
    )
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")

    header_lines = lines[:header_idx + 1]
    colnames_line = lines[header_idx + 1].rstrip("\n")

    df = pd.read_csv(
        path,
        sep=r'\s+',
        engine='python',
        skiprows=header_idx + 1
    )

    return header_lines, colnames_line, df


# -------------------------------------------------
# Detect all muscle force columns
# -------------------------------------------------
def get_all_muscle_columns(df):
    """
    In Static Optimization force .sto files,
    everything except 'time' is a muscle force.
    """
    return [c for c in df.columns if c.lower() != 'time']


# -------------------------------------------------
# Butterworth low-pass filter (zero-phase)
# -------------------------------------------------
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    wn = cutoff / nyq
    b, a = butter(order, wn, btype='low')
    return filtfilt(b, a, data, axis=0)


# -------------------------------------------------
# Main function
# -------------------------------------------------
def plot_all_muscles_filtered_percent(
    in_path,
    out_path=None,
    cutoff=20.0,
    order=4,
    plot=True
):
    """
    Filter ALL muscle forces from Static Optimization output
    and plot them vs 0–100% movement cycle.
    """

    header_lines, colnames_line, df = read_mot_with_header(in_path)

    # ---- Time and sampling frequency ----
    time_col = next(
        (c for c in df.columns if c.lower() == 'time'),
        df.columns[0]
    )
    time = df[time_col].values
    dt = np.mean(np.diff(time))
    fs = 1.0 / dt

    # ---- Normalize time to 0–100% ----
    percent = 100 * (time - time[0]) / (time[-1] - time[0])

    # ---- Muscle columns ----
    muscle_cols = get_all_muscle_columns(df)
    print(f"Filtering and plotting {len(muscle_cols)} muscles")

    # ---- Apply filtering ONLY to muscle forces ----
    muscle_data = df[muscle_cols].values
    muscle_filt = butter_lowpass_filter(
        muscle_data,
        cutoff=cutoff,
        fs=fs,
        order=order
    )

    # ---- Store filtered data ----
    df_filt = df.copy()
    df_filt.loc[:, muscle_cols] = muscle_filt

    # ---- Optional: save filtered .sto ----
    if out_path is not None:
        with open(out_path, 'w', newline='\n') as f:
            f.writelines(header_lines)
            f.write(colnames_line + '\n')

        df_filt.to_csv(
            out_path,
            sep='\t',
            index=False,
            header=False,
            float_format='%.6f',
            mode='a'
        )
        print(f"Saved filtered file to: {out_path}")

    # ---------------- Plotting ----------------
    if plot:
        n = len(muscle_cols)
        ncols = 4
        nrows = math.ceil(n / ncols)

        fig, axes = plt.subplots(
            nrows=nrows,
            ncols=ncols,
            figsize=(ncols * 4, nrows * 2.2),
            sharex=True
        )

        axes = np.atleast_1d(axes).ravel()

        for ax, muscle in zip(axes, muscle_cols):
            ax.plot(percent, df_filt[muscle].values, linewidth=1)
            ax.set_title(muscle, fontsize=9)
            ax.set_ylabel('Force (N)')
            ax.grid(True)

        for ax in axes[len(muscle_cols):]:
            ax.set_visible(False)

        for ax in axes[:ncols]:
            ax.set_xlabel('Movement cycle (%)')

        plt.xlim(0, 100)
        plt.tight_layout()
        plt.savefig(
            f'all_muscle_forces_filtered_0_100pct_{os.path.basename(in_path)}.png',
            dpi=300
        )
        plt.show()

    return df_filt


# =================================================
# USAGE
# =================================================

in_sto = r'd:\student\MTech\test\output\subject01_StaticOptimization_force.sto'

df_filtered = plot_all_muscles_filtered_percent(
    in_sto,
    cutoff=20,        # Hz (typical for muscle forces)
    order=4,
    plot=True,
    out_path=r'd:\student\MTech\test\output\subject01_StaticOptimization_force_filtered.sto'
)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.signal import butter, filtfilt


# -------------------------------------------------
# Read OpenSim .sto/.mot file with header preserved
# -------------------------------------------------
def read_mot_with_header(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    header_idx = next(
        (i for i, l in enumerate(lines) if 'endheader' in l.lower()),
        None
    )
    if header_idx is None:
        raise ValueError("No 'endheader' found in file")

    header_lines = lines[:header_idx + 1]
    colnames_line = lines[header_idx + 1].rstrip("\n")

    df = pd.read_csv(
        path,
        sep=r'\s+',
        engine='python',
        skiprows=header_idx + 1
    )

    return header_lines, colnames_line, df


# -------------------------------------------------
# Detect all muscle force columns
# -------------------------------------------------
def get_all_muscle_columns(df):
    """
    In Static Optimization force .sto files,
    everything except 'time' is a muscle force.
    """
    return [c for c in df.columns if c.lower() != 'time']


# -------------------------------------------------
# Butterworth low-pass filter (zero-phase)
# -------------------------------------------------
def butter_lowpass_filter(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    wn = cutoff / nyq
    b, a = butter(order, wn, btype='low')
    return filtfilt(b, a, data, axis=0)


# -------------------------------------------------
# Main processing + plotting function
# -------------------------------------------------
def plot_all_muscles_filtered_0_100(
    in_path,
    out_path=None,
    cutoff=20.0,
    order=4,
    plot=True
):
    """
    Filter ALL muscle forces from Static Optimization output
    and plot them vs 0–100% movement cycle.
    """

    # ---- Read file ----
    header_lines, colnames_line, df = read_mot_with_header(in_path)

    # ---- Time and sampling frequency ----
    time_col = next(
        (c for c in df.columns if c.lower() == 'time'),
        df.columns[0]
    )
    time = df[time_col].values
    dt = np.mean(np.diff(time))
    fs = 1.0 / dt

    # ---- Normalize time to 0–100% ----
    percent = 100 * (time - time[0]) / (time[-1] - time[0])

    # ---- Detect muscle columns ----
    muscle_cols = get_all_muscle_columns(df)
    print(f"Found {len(muscle_cols)} muscles")

    # ---- Filter muscle forces only ----
    muscle_data = df[muscle_cols].values
    muscle_filt = butter_lowpass_filter(
        muscle_data,
        cutoff=cutoff,
        fs=fs,
        order=order
    )

    # ---- Store filtered data ----
    df_filt = df.copy()
    df_filt.loc[:, muscle_cols] = muscle_filt

    # ---- Optional: save filtered .sto ----
    if out_path is not None:
        with open(out_path, 'w', newline='\n') as f:
            f.writelines(header_lines)
            f.write(colnames_line + '\n')

        df_filt.to_csv(
            out_path,
            sep='\t',
            index=False,
            header=False,
            float_format='%.6f',
            mode='a'
        )

        print(f"Saved filtered file to: {out_path}")

    # ---------------- Plotting ----------------
    if plot:
        n = len(muscle_cols)
        ncols = 4
        nrows = math.ceil(n / ncols)

        fig, axes = plt.subplots(
            nrows=nrows,
            ncols=ncols,
            figsize=(ncols * 4, nrows * 2.2),
            sharex=True
        )

        axes = np.atleast_1d(axes).ravel()

        for ax, muscle in zip(axes, muscle_cols):
            ax.plot(percent, df_filt[muscle].values, linewidth=1)
            ax.set_title(muscle, fontsize=9)
            ax.set_ylabel('Force (N)')
            ax.set_xlim(0, 100)              # ✅ force 0–100%
            ax.grid(True)

        # Hide unused axes
        for ax in axes[len(muscle_cols):]:
            ax.set_visible(False)

        # X-labels
        for ax in axes[:ncols]:
            ax.set_xlabel('Movement cycle (%)')

        plt.tight_layout()
        plt.savefig(
            f'all_muscle_forces_filtered_0_100pct_{os.path.basename(in_path)}.png',
            dpi=300
        )
        plt.show()

    return df_filt


# =================================================
# USAGE
# =================================================

in_sto = r'd:\student\MTech\test\output\subject01_StaticOptimization_force.sto'

df_filtered = plot_all_muscles_filtered_0_100(
    in_path=in_sto,
    cutoff=20,   # Hz (typical for muscle forces)
    order=4,
    plot=True,
    out_path=r'd:\student\MTech\test\output\subject01_StaticOptimization_force.sto'
)


In [ ]:
import opensim as osim
import os
from pathlib import Path
scale_xml = r"d:\UG_Proj\Human Sitting to Walking Transitions\S02\scale\subject02_Setup_Scale.xml"
model = r"d:\UG_Proj\Human Sitting to Walking Transitions\model\RajagopalLa2023_LL-stw_adjustedWieghts.osim"
static = r"d:\UG_Proj\Human Sitting to Walking Transitions\S02\ExpData\Mocap\trcResults\static.trc"
print(os.path.exists(model))
print(os.path.exists(static))
print(os.path.exists(scale_xml))
# osim.Logger.setLevelString("Debug")
os.chdir(r"D:\UG_Proj\Human Sitting to Walking Transitions\S02\scale")


scale_tool = osim.ScaleTool(str(scale_xml))
scale_tool.setPathToSubject("")
scaled_model = scale_tool.getMarkerPlacer().getOutputModelFileName()
scale_tool.getGenericModelMaker().setModelFileName(model)
scale_tool.getMarkerPlacer().setMarkerFileName(static)
scale_tool.getModelScaler().setMarkerFileName(static)
mp = scale_tool.getMarkerPlacer()
scale_tool.printToXML(str(scale_xml))  # Save the possibly updated XML

scale_tool.run()

True
True
True
Output model: subject02_simbody_scaled.osim
Marker file: d:\UG_Proj\Human Sitting to Walking Transitions\S02\ExpData\Mocap\trcResults\static.trc


True

In [ ]:
# osim.Logger.setLevelString("Debug")
 #already present
scale_tool = osim.ScaleTool(scale_xml)
scale_tool.setPathToSubject("")   # if using absolute paths

gmm = scale_tool.getGenericModelMaker()
mp  = scale_tool.getMarkerPlacer()

print("MODEL:", gmm.getModelFileName())
print("MARKER:", mp.getMarkerFileName())
print("OUTPUT:", mp.getOutputModelFileName())
print("SUBJECT PATH:", scale_tool.getPathToSubject())

ok = scale_tool.run()
print("RESULT:", ok)


MODEL: ..\..\model\RajagopalLa2023_LL-stw_adjustedWieghts.osim
MARKER: ../ExpData/Mocap/trcResults/static.trc
OUTPUT: subject02_simbody_scaled.osim
SUBJECT PATH: 
RESULT: True
